## CRF model

Modelo CRF para ato de aposentadoria (com dados rotulados manualmente)

In [211]:
# !pip install sklearn_crfsuite
import pandas as pd
import sklearn_crfsuite
import nltk
from nltk.tokenize import word_tokenize

In [212]:
df= pd.read_csv('result.csv')
result = pd.DataFrame(df)
result.tokenized_text = result.tokenized_text.apply(lambda x: x.encode('ascii','ignore'))

In [213]:
result['tokenized_text'][0].decode("utf-8").replace("', '", '').replace("[' ", '').replace("']", '')

'O O O O B-NUM_AJUSTE O O O B-PROCESSO O O O B-CONTRATANTE O B-CONTRATADA I-CONTRATADA I-CONTRATADA O O O O O O O O O O O O O O B-OBJ_AJUSTE I-OBJ_AJUSTE I-OBJ_AJUSTE I-OBJ_AJUSTE I-OBJ_AJUSTE I-OBJ_AJUSTE I-OBJ_AJUSTE I-OBJ_AJUSTE I-OBJ_AJUSTE I-OBJ_AJUSTE I-OBJ_AJUSTE I-OBJ_AJUSTE I-OBJ_AJUSTE I-OBJ_AJUSTE I-OBJ_AJUSTE I-OBJ_AJUSTE I-OBJ_AJUSTE I-OBJ_AJUSTE I-OBJ_AJUSTE I-OBJ_AJUSTE I-OBJ_AJUSTE I-OBJ_AJUSTE I-OBJ_AJUSTE I-OBJ_AJUSTE I-OBJ_AJUSTE I-OBJ_AJUSTE I-OBJ_AJUSTE I-OBJ_AJUSTE I-OBJ_AJUSTE I-OBJ_AJUSTE I-OBJ_AJUSTE O O O O O O B-VALOR O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O B-PT I-PT O O O O O O O O O O O O O O O O O O O B-VIGENCIA I-VIGENCIA I-VIGENCIA I-VIGENCIA I-VIGENCIA I-VIGENCIA I-VIGENCIA I-VIGENCIA I-VIGENCIA I-VIGENCIA I-VIGENCIA I-VIGENCIA I-VIGENCIA I-VIGENCIA I-VIGENCIA I-VIGENCIA I-VIGENCIA I-VIGENCIA I-VIGENCIA I-VIGENCIA O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O'

Load csv with text and labels

In [214]:
x = []
y = []
for row in range(len(result)):
    if pd.notna(result['treated_text'][row]):
        text_iob = result['tokenized_text'][row].decode("utf-8").replace("', '", '').replace("[' ", '').replace("']", '')
        x.append(word_tokenize(result['treated_text'][row]))
        y.append(text_iob.split())
len(x), len(y)

(1168, 1168)

In [215]:
# Tirando os rotulos que nao batem com os dados
continua = True
while continua == True:
    continua = False
    for i in range(len(x)):
        if len(x[i]) > len(y[i]):
            x[i].pop()
            continua = True

        if len(x[i]) < len(y[i]):
            y[i].pop()
            continua = True

Create dictionary feature for each word in each sequence in x

In [216]:
def get_features(sentence):
        """Create features for each word in act.
        Create a list of dict of words features to be used in the predictor module.
        Args:
            act (list): List of words in an act.
        Returns:
            A list with a dictionary of features for each of the words.
        """
        sent_features = []
        for i in range(len(sentence)):
            # print(sentence[i])
            word_feat = {
                'word': sentence[i].lower(),
                'capital_letter': sentence[i][0].isupper(),
                'all_capital': sentence[i].isupper(),
                'isdigit': sentence[i].isdigit(),
                'word_before': sentence[i].lower() if i == 0 else sentence[i-1].lower(),
                'word_after:': sentence[i].lower() if i+1 >= len(sentence) else sentence[i+1].lower(),
                'BOS': i == 0,
                'EOS': i == len(sentence)-1
            }
            sent_features.append(word_feat)
        return sent_features
    
for i in range(len(x)):
    x[i] = get_features(x[i])

Separate train and test splits (in order)

In [217]:
import math
split_idx = math.floor(0.8*len(x))
x_train = x[0:split_idx]
y_train = y[0:split_idx]
x_test = x[split_idx:]
y_test = y[split_idx:]
len(x_train),  len(x_test), len(y_train)


(934, 234, 934)

In [228]:
import sklearn_crfsuite
from sklearn_crfsuite import metrics

crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs', 
    c1=0.1, 
    c2=0.1, 
    max_iterations=100, 
    all_possible_transitions=True
)

crf.fit(x_train, y_train)

CRF(algorithm='lbfgs', all_possible_transitions=True, c1=0.1, c2=0.1,
    keep_tempfiles=None, max_iterations=100)

In [229]:
classes = list(crf.classes_)
classes.remove('O')
print(classes)
y_pred = crf.predict(x_test)
metrics.flat_f1_score(y_test, y_pred, average='weighted', labels=classes)

['B-NUM_AJUSTE', 'B-PROCESSO', 'B-CONTRATANTE', 'B-CONTRATADA', 'I-CONTRATADA', 'B-OBJ_AJUSTE', 'I-OBJ_AJUSTE', 'B-VALOR', 'B-PT', 'I-PT', 'B-VIGENCIA', 'I-VIGENCIA', 'I-CONTRATANTE', 'B-DATA_ASSINATURA', 'I-DATA_ASSINATURA', 'B-NE', 'B-CODIGO_UO', 'B-ND', 'B-FUND_DISPENSA', 'I-FUND_DISPENSA', 'B-MODALIDADE_LICITACAO', 'I-MODALIDADE_LICITACAO', 'B-NUM_LICITACAO', 'B-TIPO_OBJ', 'I-TIPO_OBJ', 'B-ORGAO_LICITANTE', 'B-DATA_ABERTURA', 'I-DATA_ABERTURA', 'B-CODIGO_SISTEMA_COMPRAS', 'B-VALOR_ESTIMADO', 'B-NUM_CONTRATO', 'B-OBJ_ADITIVO', 'I-OBJ_ADITIVO', 'I-ND', 'B-NUM_ADITIVO', 'I-NUM_ADITIVO', 'B-IDENT_REVOGACAO_ANULACAO', 'I-IDENT_REVOGACAO_ANULACAO', 'B-SISTEMA_COMPRAS', 'I-VALOR_ESTIMADO', 'I-SISTEMA_COMPRAS', 'B-CONVENENTE', 'I-CONVENENTE', 'B-IDENT_DISPENSA', 'B-OBJ_LICITACAO', 'I-OBJ_LICITACAO', 'I-PROCESSO', 'I-NUM_LICITACAO', 'I-ORGAO_LICITANTE', 'I-VALOR', 'I-NE', 'I-NUM_CONTRATO', 'B-DECISAO_TCDF', 'I-IDENT_DISPENSA', 'I-DECISAO_TCDF', 'B-PRAZO_SUSPENSAO', 'I-PRAZO_SUSPENSAO', 'I-C

0.6804841275892826

In [230]:
print(metrics.flat_classification_report(
    y_test, y_pred, labels=classes, digits=3
))

                            precision    recall  f1-score   support

              B-NUM_AJUSTE      0.200     0.009     0.017       116
                B-PROCESSO      1.000     0.023     0.045       130
             B-CONTRATANTE      0.817     0.694     0.751       206
              B-CONTRATADA      0.486     0.148     0.227       115
              I-CONTRATADA      0.552     0.188     0.281       536
              B-OBJ_AJUSTE      0.654     0.440     0.526       116
              I-OBJ_AJUSTE      0.751     0.645     0.694      3781
                   B-VALOR      0.154     0.030     0.051        66
                      B-PT      0.286     0.035     0.062        57
                      I-PT      0.000     0.000     0.000         4
                B-VIGENCIA      0.651     0.500     0.566       112
                I-VIGENCIA      0.757     0.836     0.794      1413
             I-CONTRATANTE      0.962     0.864     0.910       927
         B-DATA_ASSINATURA      0.154     0.018